#### @prompt example

## LangChain with OpenBB
This notebook will leverage the following technologies/frameworks to do some stock research.
It will create an Agent that will fetch information for top performing companies in the top performing industry
It will then leverage Chroma to load the latest earning transcript call for Altria (MO) to allow users to ask few questions
on Altria's latest result.
This work was inspired by this article

- Gemini
- LangChain
- OpenBB (openbb.co)

In [2]:
!pip install langchain
!pip install -U langchain-google-genai
!pip install -U -q "google-genai==1.7.0"
!pip install langchain_community
!pip install docx2txt
!pip install chromadb
!pip install wikipedia
!pip install finvizfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigtable 2.27.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
google-generativeai 0.8.3 requires google-ai-generativelanguage==0.6.10, but you have google-ai-generativelanguage 0.6.17 which is incompatible.
opentelemetry-proto 1.32.0 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0,

In [3]:
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [73]:
from langchain_core.tools import tool

@tool
def get_sectors_performance(sector:str = None):
    """ Useful for getting performance of each  sector for last week, last month, last quarter, last half year and last year. **This tool does not require any input from the user.**"""
    
    from finvizfinance.group import Performance

    try:
        performance = Performance()
        # Get the performance data
        return performance.screener_view().to_dict('records')
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure the finvizfinance library is installed correctly.")
        print("You can install it using: pip install finvizfinance")
        print("Also, check your internet connection as the library fetches data from Finviz.")

@tool
def get_companies_for_sector(sectorName:str):
    """ Return a subset of companies for the given sector"""
    from finvizfinance.screener.overview import Overview
    foverview = Overview()
    filters_dict = {'Sector': sectorName,
                    'Market Cap.': '+Small (over $300mln)',
                    'Average Volume': 'Over 200K',
                    'Current Ratio': 'Over 1',
                    'Debt/Equity': 'Under 1',
                    'InstitutionalOwnership': 'Under 60%',
                    'Price': 'Over $10'}
    foverview.set_filter(filters_dict=filters_dict)
    df = foverview.screener_view(order='Company')
    return df.head(5).to_dict('records')


get_companies_for_sector.invoke('Basic Materials')

[{'Ticker': 'MT',
  'Company': 'ArcelorMittal',
  'Sector': 'Basic Materials',
  'Industry': 'Steel',
  'Country': 'Luxembourg',
  'Market Cap': 20600000000.0,
  'P/E': 16.15,
  'Price': 26.81,
  'Change': 0.052000000000000005,
  'Volume': 2371168.0},
 {'Ticker': 'BHP',
  'Company': 'BHP Group Limited ADR',
  'Sector': 'Basic Materials',
  'Industry': 'Other Industrial Metals & Mining',
  'Country': 'Australia',
  'Market Cap': 115150000000.0,
  'P/E': 10.15,
  'Price': 45.38,
  'Change': 0.0435,
  'Volume': 3227365.0},
 {'Ticker': 'CENX',
  'Company': 'Century Aluminum Co',
  'Sector': 'Basic Materials',
  'Industry': 'Aluminum',
  'Country': 'USA',
  'Market Cap': 1460000000.0,
  'P/E': 4.9,
  'Price': 15.77,
  'Change': 0.07719999999999999,
  'Volume': 1233176.0},
 {'Ticker': 'BVN',
  'Company': 'Compania de Minas Buenaventura S.A. ADR',
  'Sector': 'Basic Materials',
  'Industry': 'Other Precious Metals & Mining',
  'Country': 'Peru',
  'Market Cap': 3750000000.0,
  'P/E': 9.32,
  

In [59]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

from langchain.agents import create_structured_chat_agent, AgentExecutor

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain




### Chat Memory

In [60]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentType, initialize_agent

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock recommendation assistant , but dont know current events so you should use your tools as much as you can.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [66]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


# Initialize Gemini Pro model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7
)


# Define the tools and create a "tools" node.

tools = [get_sectors_performance, get_companies_for_sector ]

# Attach the tools to the model so that it knows what it can call.
#llm_with_tools = llm.bind_tools(tools)

In [68]:
chat_history = []
chat_history.append(HumanMessage(content="Your question here"))
chat_history.append(AIMessage(content="AI response here"))
# Define your prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="chat_history"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, max_token_limit=18000)

# Initialize the agent
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

### Finding the best performing sector across week, month, quarter, and year

In [70]:
input1 = '''Find the sector which across week, month, quarter and year has shown the best performance and summarize it.
'''
result = agent_chain.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: get_sectors_performance
Action Input: 
```
Observation: [{'Name': 'Basic Materials', 'Perf Week': '6.89%', 'Perf Month': -0.0282, 'Perf Quart': 0.011699999999999999, 'Perf Half': -0.10949999999999999, 'Perf Year': -0.09820000000000001, 'Perf YTD': 0.0124, 'Recom': 1.99, 'Avg Volume': 692160000.0, 'Rel Volume': 2.28, 'Change': 0.038599999999999995, 'Volume': 1290000000.0}, {'Name': 'Communication Services', 'Perf Week': '5.44%', 'Perf Month': -0.0407, 'Perf Quart': -0.0823, 'Perf Half': -0.0070999999999999995, 'Perf Year': 0.0711, 'Perf YTD': -0.0785, 'Recom': 1.62, 'Avg Volume': 720630000.0, 'Rel Volume': 0.73, 'Change': 0.0095, 'Volume': 429440000.0}, {'Name': 'Consumer Cyclical', 'Perf Week': '2.99%', 'Perf Month': -0.0451, 'Perf Quart': -0.142, 'Perf Half': -0.0718, 'Perf Year': 0.0034000000000000002, 'Perf YTD': -0.1543, 'Recom': 1.8, 'Avg Volume': 1670000000.0, 'Rel Volume': 1.49, 'Change': 0

### Now we are going to find few companies for the best performing sector. We'll output a table sorted by P/E

In [77]:
input1 = "Now find me some companies for the sector you found in the previous step. Create  a table with Ticker, Company,  P/E and change"
result = agent_chain.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: | Ticker | Company                             | P/E   | Change |
| ------ | ----------------------------------- | ----- | ------ |
| AFL    | Aflac Inc                           | 11.08 | 0.0169 |
| BRK-B  | Berkshire Hathaway Inc              | 12.69 | 0.0159 |
| ACT    | Enact Holdings Inc                  | 8.06  | 0.0109 |
| FIHL   | Fidelis Insurance Holdings Ltd      | 16.73 | -0.0023 |
| GGAL   | Grupo Financiero Galicia ADR        | 5.11  | 0.0965 |

> Finished chain.
| Ticker | Company                             | P/E   | Change |
| ------ | ----------------------------------- | ----- | ------ |
| AFL    | Aflac Inc                           | 11.08 | 0.0169 |
| BRK-B  | Berkshire Hathaway Inc              | 12.69 | 0.0159 |
| ACT    | Enact Holdings Inc                  | 8.06  | 0.0109 |
| FIHL   | Fidelis Insurance Holdings Ltd      | 16.73 | -0.0023 |
| GGAL   | Grupo Financiero Galicia ADR 

### Now we will play around with Chroma
### We will upload an extract from Altria Group latest Earning calls and ask our llm to extract some information

In [81]:
with open('/kaggle/input/altria-q424-earning-call/Altria_Q424EarningCall.txt') as f:
    for l in f.readlines():
        print(l)

[

  {

    "symbol": "MO",

    "quarter": 4,

    "year": 2024,

    "date": "2025-01-30 09:00:00",

    "content": "Operator: Good day. And welcome to the Altria Group 2024 Fourth Quarter and Full Year Earnings Conference Call. Today's call is scheduled to last about one hour, including remarks by Altria's management and question-and-answer session. [Operator instructions] I would now like to turn the call over to Mac Livingston, vice president of investor relations for Altria Client Services. Please go ahead, sir.\nMac Livingston: Thanks Shelby. Good morning, and thank you for joining us. This morning, Billy Gifford, Altria's CEO, and Sal Mancuso, our CFO, will discuss Altria's fourth quarter and full year business results. Earlier today, we issued a press release providing our results. The release, presentation, quarterly metrics, and our latest corporate responsibility reports are all available at altria.com. During our call today, unless otherwise stated, we're comparing results